In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
total_data=pd.read_csv('income2023f/totaldata.csv')

In [ ]:
total_data = total_data.iloc[: , 1:]

In [ ]:
train_datam=total_data[total_data['income>50K']!=-999]
test_datam=total_data[total_data['income>50K']==-999]
train_datam.loc[train_datam['income>50K']==0,['income>50K']]=-1

<ipython-input-39-68a8aa734ef9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_datam.loc[train_datam['income>50K']==0,['income>50K']]=-1


In [ ]:
train_data=train_datam.iloc[:20000]
test_data=train_datam.iloc[20000:]
test_data=test_data.reset_index(drop=True)

In [ ]:
train_data

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income>50K
0,53,1,93449,1,15,1,1,1,1,1,0,0,40,1,1
1,33,1,123424,2,13,1,2,1,2,1,0,0,40,2,1
2,47,2,144844,3,9,1,3,1,2,1,0,0,40,2,-1
3,40,2,114580,3,9,2,3,2,2,2,0,0,40,3,-1
4,39,2,115618,3,9,1,4,1,2,1,0,0,50,2,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,36,2,174242,2,13,3,2,4,2,1,0,0,60,2,-1
19996,23,2,179241,13,11,3,7,5,2,2,0,0,40,2,-1
19997,32,2,235124,8,8,2,5,4,2,1,0,0,30,2,-1
19998,33,1,103435,1,15,1,1,1,2,1,7688,0,50,2,1


In [ ]:
def standard_perceptron(train_data,r,T=10):
  w=np.zeros(train_data.iloc[0,:-1].shape[0]+1)
  for j in range(T):
    train_data=train_data.sample(frac = 1)
    y=train_data.iloc[:,-1]
    x=train_data.iloc[:,:-1]
    x=x.to_numpy()
    y=y.to_numpy()
    x=np.insert(x, 0, 1, axis=1)
    for i in range(len(x)):
      if (y[i]*(np.dot(w,x[i])))<=0:
        w+=r*y[i]*x[i]
  return w


In [ ]:
def voted_perceptron(train_data,r,T=10):
  w=np.zeros(train_data.iloc[0,:-1].shape[0]+1)
  bag=[]
  cm=0
  for j in range(T):
    train_data=train_data.sample(frac = 1)
    y=train_data.iloc[:,-1]
    x=train_data.iloc[:,:-1]
    x=x.to_numpy()
    y=y.to_numpy()
    x=np.insert(x, 0, 1, axis=1)
    for i in range(len(x)):
      if (y[i]*(np.dot(w,x[i])))<=0:
        bag.append((w.copy(),cm))
        w+=r*y[i]*x[i]
        cm=1
      else:
        cm+=1
  bag.append((w.copy(),cm))
  return bag

In [ ]:
def averaged_perceptron(train_data,r,T=10):
  w=np.zeros(train_data.iloc[0,:-1].shape[0]+1)
  a=np.zeros(train_data.iloc[0,:-1].shape[0]+1)
  for j in range(T):
    train_data=train_data.sample(frac = 1)
    y=train_data.iloc[:,-1]
    x=train_data.iloc[:,:-1]
    x=x.to_numpy()
    y=y.to_numpy()
    x=np.insert(x, 0, 1, axis=1)
    for i in range(len(x)):
      if (y[i]*(np.dot(w,x[i])))<=0:
        w+=r*y[i]*x[i]
      a+=w
  return a

In [ ]:
def prediction_standard_averaged(test_data,w):
  y=test_data.iloc[:,-1]
  x=test_data.iloc[:,:-1]
  x=x.to_numpy()
  y=y.to_numpy()
  x=np.insert(x, 0, 1, axis=1)
  pred=np.dot(x,w)
  pred_s=np.sign(pred)
  try:
   diff=np.where((pred_s*y)==-1,1,0)
  except:
    diff=np.zeros_like(y)
  return(np.sum(diff)/diff.shape[0]),pred_s

In [ ]:
def prediction_voted(test_data,bag):
  y=test_data.iloc[:,-1]
  x=test_data.iloc[:,:-1]
  x=x.to_numpy()
  y=y.to_numpy()
  pred_tot=np.zeros_like(y)
  x=np.insert(x, 0, 1, axis=1)
  for i in bag:
    w=i[0]
    c=i[1]
    pred=np.dot(x,w)
    pred_c=c*np.sign(pred)
    pred_tot=np.add(pred_tot,pred_c)
  pred_s=np.sign(pred_tot)
  diff=np.where((pred_s*y)==-1,1,0)
  return(np.sum(diff)/diff.shape[0]),pred_s

In [ ]:
w1=standard_perceptron(train_data,1e-5)

In [ ]:
w2=voted_perceptron(train_data,1e-5)

In [ ]:
w3=averaged_perceptron(train_data,1e-5)

In [ ]:
w1

array([-1.406000e-02,  1.638270e+00,  4.516000e-02, -3.952290e+00,
       -4.336800e-01,  4.751000e-01, -3.719200e-01, -4.205400e-01,
       -4.542900e-01, -6.102000e-02, -9.432000e-02,  4.068965e+01,
        3.634720e+01,  1.499480e+00, -1.069200e-01])

In [ ]:
w2

[(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 0),
 (array([-1.00000e-05, -2.70000e-04, -2.00000e-05, -1.68827e+00,
         -2.00000e-05, -1.30000e-04, -3.00000e-05, -8.00000e-05,
         -5.00000e-05, -2.00000e-05, -1.00000e-05,  0.00000e+00,
          0.00000e+00, -2.00000e-05, -2.00000e-05]),
  3),
 (array([ 0.0000e+00,  3.3000e-04,  3.0000e-05,  1.2369e-01,  4.0000e-05,
         -3.0000e-05, -2.0000e-05, -6.0000e-05, -4.0000e-05,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  3.8000e-04,  0.0000e+00]),
  1),
 (array([-1.00000e-05,  9.00000e-05,  1.00000e-05, -2.73406e+00,
          2.00000e-05, -1.60000e-04, -5.00000e-05, -1.30000e-04,
         -8.00000e-05, -2.00000e-05, -1.00000e-05,  0.00000e+00,
          0.00000e+00, -2.00000e-05, -2.00000e-05]),
  1),
 (array([ 0.0000e+00,  3.7000e-04,  3.0000e-05, -1.0607e+00,  5.0000e-05,
         -7.0000e-05, -4.0000e-05, -1.1000e-04, -2.0000e-05,  0.0000e+00,
          1.0000e-05,  0.0000e+00,  0.0

In [ ]:
w3

array([ 3.13953   , -3.68232469, -2.43041032, -2.52316705, -0.75364622])

In [ ]:
prediction_standard_averaged(test_data,w1)

(0.2238, array([-1., -1., -1., ..., -1., -1., -1.]))

In [ ]:
a,b=prediction_standard_averaged(test_datam,w3)

In [ ]:
b.shape

(23842,)

In [ ]:
g,h=prediction_voted(test_datam,w2)

In [ ]:
import csv
a=list(range(1,23843))
with open('/content/drive/MyDrive/income2023f/percepvoted.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(a,h))